# Assignment 1
## 1. Assignment1은 위에서 언급 되었던 Multiclass SVM을 직접 구현하시는 것입니다. 
기본적으로 사이킷 런에 있는 SVM은 멀티클래스 SVM을 지원합니다. 그러나 과제에서는 그것을 쓰면 안됩니다! 아이리스 데이터는 총 세 개의 클
래스가 있으므로 이 클래스를 one-hot 인코딩 한 뒤, 각각 binary SVM을 트레이닝하고 이 결과를 조합하여
multiclass SVM을 구현하는 것입니다.

## 2. 위에서 말했듯 기본적으로 one vs one, one vs rest방법이 있으며 어떤 것을 구현하든 자유입니다.
만약 투표결과 동점이 나온경우(예를 들어 각각의 SVM의 결과가 A vs B C의 경우 A로 판별, B vs A C의 결과 B로 판별, C vs A
B의 경우 C로 판별한 경우 투표를 통해 class를 결정할 수 없음) decision_function을 활용하시거나, 가장 개수가
많은 클래스를 사용하시거나 랜덤으로 하나를 뽑거나 하는 방법 등을 이용해 동점자인 경우를 판별해주시면 됩니다.
공식 문서를 보면 사이킷런이 어떤 방법으로 구현했는지가 글로 나와 있으므로 참조하셔도 무관합니다.

## 3. 과제코드에는 제가 iris 데이터를 로드하고 iris 데이터를 one hot인코딩 한 부분까지 구현해 놓았습니다.
또한 decision function을 호출해서 사용하는 예시도 하나 넣어 놓았으니 참고하시면 됩니다.
개인적으로 one vs rest가 더 구현하기 쉬울것으로 생각되며, 모르는 부분은 언제든 질문해주세요! 생각보다 코드가
길지 않고 어렵지 않습니다.

iris데이터를 불러와 target 변수를 원래데이터와 나누고 X,y라는 이름으로 지정해 주었습니다. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

iris =  sns.load_dataset('iris')
X= iris.iloc[:,:4]
y = iris.iloc[:,-1]
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
scaler = StandardScaler() 
X = scaler.fit_transform(X)
#정규화를 시켜주도록 하였습니다. 

- One Versus Rest방식을 사용하여 svm_1,2,3로 3개로 나누고 각 종류를 분류해주는 기계를 만들도록 하였습니다.
- gamma가 클수록 인접한 것들에 대해서만 같은 영역으로 판단하게 됩니다.또한 gamma가 너무 크면 overfitting의 가능성이 있습니다. 따라서 grid_search를 활용하여 적절한 gamma와 c값을 찾아보고자 하였습니다. 

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipe_svc = Pipeline([('scl', StandardScaler()), ('clf', SVC(random_state=1))])

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {'clf__C': param_range, 'clf__kernel': ['linear']},
    {'clf__C': param_range, 'clf__gamma': param_range, 'clf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', cv=10, n_jobs=1)
%time gs = gs.fit(X, y)

Wall time: 5.22 s


In [9]:
print(gs.best_score_)
print(gs.best_params_)

0.9800000000000001
{'clf__C': 1.0, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}


In [10]:
svm_1 = SVC(kernel ='rbf', gamma = 0.1, C = 1)
svm_2 = SVC(kernel ='rbf', gamma = 0.1, C = 1)
svm_3 = SVC(kernel ='rbf', gamma = 0.1, C = 1)
#결과값에 따라서 svm_1,2,3라는 이름으로 분류 머신 3가지를 만들었습니다. 

In [13]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

#위와같은 이름으로 train,test set을 분류하였습니다. 

In [14]:
y_train = pd.get_dummies(y_train) 
y_test = pd.get_dummies(y_test)
#one hot encoding방식을 통해 종류에 따라서 해당하는 종류일때 1이 나오도록 인코딩 하였습니다. 

In [15]:
svm_1.fit(X_train,y_train.iloc[:,0])
svm_2.fit(X_train,y_train.iloc[:,1])
svm_3.fit(X_train,y_train.iloc[:,2])
print(svm_1.decision_function(X_test)) 
print(svm_2.decision_function(X_test))
print(svm_3.decision_function(X_test)) 
#각기계별로 decision_function hyperplane과의 거리를 확인해 보도록 하겠습니다. 

[-1.18554588 -1.34769197 -1.57406849  0.03401689 -1.31124945 -1.51399498
  1.39065218 -1.47977585  1.22193286 -1.41059969 -1.47581591  1.30551737
  1.19104597 -1.58535794 -1.4738944  -1.18464392  1.3442136  -1.34328399
 -1.57373443 -1.38712859  0.90326855 -1.52423684 -1.44461302 -1.2988055
 -1.34222782  1.44961849  1.12163393 -1.32025178 -1.57159424 -1.03725897]
[ 1.07355283  1.12326184  0.22014949 -0.24698463  0.12662226 -0.32322719
 -1.58289696  0.1224296  -1.63491391  1.01335926 -0.25906672 -1.1018622
 -0.836975    0.0302752   0.70097662  0.49791592 -1.45738506  1.1268945
  0.3277328  -1.36922772 -1.17329127 -0.55671024  0.11751682  0.649796
 -0.32987709 -1.61111572 -0.75933661 -1.27752528 -1.32513305  1.00806717]
[-1.30254712 -1.1187129   0.02848374 -1.54546193 -0.16851305  0.51385959
 -1.73132876  0.01337511 -1.22691105 -0.98071497  0.42073173 -2.20142821
 -2.35970574  0.21222381 -0.57524412 -0.63574206 -1.97107645 -1.13905868
 -0.09658363  1.41789714 -1.76544071  0.74248969  0.01

부호가 모두 같은 경우, 즉, 다른 분류기계이지만 같다고 판단하는 경우 == 동점인 경우라고 판단 할 수 있습니다. 이경우에는 적절하게 판단했다라고 볼 기계를 한가지 정하는 방법을 사용할 수 있습니다.
또한 어떤 기계가 더 적절한지는 hyperplane과의 거리 확인을 통해서 거리가 멀경우 그 기계가 그나마 더 판단을 잘한다라고 생각 해 볼 수 있을 것입니다. 

In [19]:
for i in range(len(X_test)):
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i]))\
    and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        print(i)
#아무런 출력값이 없는 것으로 보아 다행이 그러한 경우는 없는 것으로 판단할 수 있습니다. (이전에는 [3],[17],[18]의 경우가 있었지만 추가적으로 grid_Search를 진행하면서 이러한 경우가 사라진 것으로 판단됩니다.)

In [20]:
#아쉬운대로 이전에 이러했던 경우에 대해서 hyperplane을 확인해보도록 하겠습니다.
print(svm_1.decision_function(X_test)[3])
print(svm_2.decision_function(X_test)[3])
print(svm_3.decision_function(X_test)[3])
print("====")
print(svm_1.decision_function(X_test)[17])
print(svm_2.decision_function(X_test)[17])
print(svm_3.decision_function(X_test)[17])
print("====")
print(svm_1.decision_function(X_test)[18])
print(svm_2.decision_function(X_test)[18])
print(svm_3.decision_function(X_test)[18])

0.0340168856423895
-0.2469846319398301
-1.54546193457456
====
-1.3432839922630626
1.1268944982050129
-1.139058680003137
====
-1.5737344322689866
0.3277327994150263
-0.09658363241711539


In [27]:
print(svm_1.predict(X_test))
print(svm_2.predict(X_test))
print(svm_3.predict(X_test))

[0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[1 1 1 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1]
[0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0]


In [28]:
score_1 = svm_1.score(X_test,y_test.iloc[:,0])
score_2 = svm_2.score(X_test,y_test.iloc[:,1])
score_3 = svm_3.score(X_test,y_test.iloc[:,2])
print(score_1)
print(score_2)
print(score_3)

1.0
0.9
0.9666666666666667


이전과는 다르게 score가 매우 향상된 것을 확인할 수 있었습니다. 이럴경우 근소하게 1에 가까운 SVM_1머신을 좀 더 믿을만 하다고 판단해 볼 수 도있지만 3가지 모두 점수가 매우 높기 때문에 어떤 것을 사용하더라도 매우 우수한 결과가 나올 것으로 판단 할 수 있습니다. 